In [1]:
import unittest
import nbimporter
import pandas as pd
import pulp
import Travelling_Salesman as tsp

Importing Jupyter notebook from Travelling_Salesman.ipynb


In [2]:
S = tsp.Space()

In [3]:
S.nodes = {'A': [70, 31], 
           'B': [15, 34], 
           'C': [30, 89], 
           'D': [53, 64]}

In [4]:
S.nodesToDF()

,x,y
C,30,89
A,70,31
D,53,64
B,15,34


In [5]:
S.getDistances()

,C,A,D,B
C,100.000000,70.455660,33.970576,57.008771
A,70.455660,100.000000,37.121422,55.081757
D,33.970576,37.121422,100.000000,48.414874
B,57.008771,55.081757,48.414874,100.000000


### Space Methods

In [6]:
class TestSpaceMethods(unittest.TestCase):
    
    def test_posToChar(self):
        self.assertEqual(S.posToChar(25),'Z')
        
    def test_intToChar(self):
        self.assertEqual(S.intToChar(51),'AZ')
    
    def test_nodesToDF(self):
        self.assertEqual(S.nodesToDF().sort_index().loc['A','x'],70)
    
    def test_getDistances(self):
        S.getDistances()
        self.assertEqual(S.totalDistances[('A','A')],0)
    
    def test_getDistance(self):
        self.assertEqual(S.getDistance(('A','B'),2,0),55.08175741568164)
        
    def test_getTotalDistance(self):
        self.assertEqual(S.getTotalDistance(('A','B','C','D')),183.18252656010577)
    

### Naive

In [7]:
N = tsp.Naive()

In [8]:
S.permutations = [['A','B'],['C','D']]

In [9]:
class TestNaiveMethods(unittest.TestCase):

    def test_permutation(self):
        self.assertEqual(N.permutation(['A','B','C']),
                        [['A', 'B', 'C'],
                         ['A', 'C', 'B'],
                         ['B', 'A', 'C'],
                         ['B', 'C', 'A'],
                         ['C', 'A', 'B'],
                         ['C', 'B', 'A']])
    
    def test_getBest(self):
        self.assertEqual(N.getBest(S,False),['C','D'])


### Greedy

In [10]:
G = tsp.Greedy()

In [11]:
class TestGreedyMethods(unittest.TestCase):

    def test_getMinRowCol(self):
        self.assertEqual(G.getMinRowCol(S.distances),('C', 'D'))


### Optimization

In [12]:
O = tsp.Optimization()
model = pulp.LpProblem("TSP", pulp.LpMinimize)

In [13]:
class TestOptimizationMethods(unittest.TestCase):

    def test_identifySubtourFromTuples(self):
        self.assertEqual(O.identifySubtourFromTuples(S,[('A', 'B'), ('B', 'A'),
                                                        ('C', 'D'), ('D', 'C'),
                                                        ('E', 'F'), ('F', 'E')]),
                         
                                                     [[('A', 'B'), ('B', 'A')], 
                                                      [('C', 'D'), ('D', 'C')], 
                                                      [('E', 'F'), ('F', 'E')]])
        
    def test_getColInd(self):
        self.assertEqual(O.getColInd(('A','B'),S),(1,3))
        
    def test_tupleToSolution(self): 
        self.assertEqual(O.tupleToSolution([[('A', 'F'), ('F', 'C'), 
                                             ('C', 'A'),('B', 'D'), 
                                             ('D', 'E'), ('E', 'B')]]),
                         
                                            ['A', 'F', 'C', 'B', 'D', 'E'])
        

In [14]:
unittest.main(argv=['first-arg-is-ignored'], exit=False)

............
----------------------------------------------------------------------
Ran 12 tests in 0.023s

OK
